In [1]:
# Imports
import pandas as pd
import numpy as np
from scipy.stats import norm, uniform
from itertools import combinations
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import trange
from shared.config_plt import config_plt, get_fig_ax, save_my_fig, colors
from shared.utils import print_se_parentheses, rank
from shared.univariate import DiscretizedDist, PriorAnalyzer, RankSumAnalyzer, RoundedAnalyzer, NeighborhoodAnalyzer, BRIAsympAnalyzer, DiffMeansAnalyzer, LIBDiffMeansAnalyzer
theta_rng = np.random.default_rng(seed=1)
data_rng = np.random.default_rng(seed=2)
config_plt()

In [2]:
# Setup
theta_abs_lim = 50.
theta_min = -theta_abs_lim
theta_max = theta_abs_lim
theta_inc = 0.01
theta_vals = np.arange(theta_min, theta_max+0.0001, theta_inc)
n_theta_vals = theta_vals.size
alpha = 0.05
n_each_vals = np.array([10, 40, 200, 1000])
n_sample_sizes = n_each_vals.size

prior_dist = DiscretizedDist(norm(loc=0., scale=10.), theta_vals, theta_rng)
methods = [
    PriorAnalyzer("Prior", alpha, prior_dist),
    DiffMeansAnalyzer("DIM", alpha, prior_dist),
    LIBDiffMeansAnalyzer("LIB", alpha, prior_dist),
    BRIAsympAnalyzer("BRI-A", alpha, prior_dist),
]
method_names = [m.name for m in methods]
n_methods = len(methods)

In [3]:
# Simulation
n_reps = 10000
posterior_means = np.zeros((n_reps, n_sample_sizes, n_methods))
lower_bounds = np.zeros((n_reps, n_sample_sizes, n_methods))
upper_bounds = np.zeros((n_reps, n_sample_sizes, n_methods))
nominal_coverage_rates = np.zeros((n_reps, n_sample_sizes, n_methods))
true_thetas = np.zeros((n_reps, n_sample_sizes))

def sample_a(n_arange, n_each):
    a_idx = data_rng.choice(n_arange, n_each, replace=False)
    a_sample = np.isin(n_arange, a_idx)
    return a_sample

# Simulation
for i in trange(n_reps):
    for k, n_each in enumerate(n_each_vals):
        # Generate data
        n = 2*n_each
        n_arange = np.arange(n)
        a_init = sample_a(n_arange, n_each)
        not_a_init = ~a_init
        y0_init = data_rng.normal(size=n, scale=10.) + data_rng.gamma(20./5., scale=5./2., size=n)
        y1_init = y0_init + 10. + data_rng.normal(size=n, scale=10.)
        ya_init = a_init*y1_init + not_a_init*y0_init
        true_theta_i = prior_dist.rvs()
        y0 = ya_init - a_init*true_theta_i
        y1 = y0 + true_theta_i
        true_thetas[i, k] = true_theta_i
        a = sample_a(n_arange, n_each)
        not_a = ~a
        y = a*y1 + not_a*y0

        for j, method in enumerate(methods):
            # Get posterior probs
            pm, lb, ub, cr = method.analyze(y, a, theta_vals)
            posterior_means[i, k, j] = pm
            lower_bounds[i, k, j] = lb
            upper_bounds[i, k, j] = ub
            nominal_coverage_rates[i, k, j] = cr

100%|█████████████████████████████████████| 10000/10000 [30:07<00:00,  5.53it/s]


In [8]:
# Get mean and se
def get_mean_and_se(x, digits=3, stars=False, alpha=0.05):
    mean = x.mean(axis=0)
    se = x.std(axis=0) / np.sqrt(x.shape[0])
    mean_se = print_se_parentheses(mean, se, digits=digits, stars=stars, alpha=alpha)
    return mean_se

# Process results
errors = (posterior_means.T - true_thetas.T).T
errors2 = errors**2
ci_covered = ((lower_bounds.T <= true_thetas.T) & (true_thetas.T <= upper_bounds.T)).T
ci_length = upper_bounds - lower_bounds

arrs_to_process = [errors, errors2, ci_covered, nominal_coverage_rates, ci_length]
arrs_names_series = pd.Series(["Bias", "MSE", "CI Coverage", "CI Nominal Level", "CI Length"], name="Metrics")
method_names_series = pd.Series(method_names, name="Methods")

results_df = pd.DataFrame(
    np.vstack([
        np.stack([get_mean_and_se(arr[:, i]) for i in range(n_sample_sizes)])
        for arr in arrs_to_process]),
    index=pd.MultiIndex.from_product([arrs_names_series, n_each_vals.astype(str)]),
    columns=method_names_series)
results_df

Methods                          Prior             DIM             LIB  \
Metrics                                                                  
Bias             10      0.062 (0.099)  -0.041 (0.067)   0.406 (0.058)   
                 40     -0.143 (0.100)  -0.023 (0.033)   0.155 (0.032)   
                 200     0.130 (0.100)  -0.004 (0.015)   0.047 (0.015)   
                 1000    0.068 (0.100)   0.006 (0.007)   0.016 (0.007)   
MSE              10     98.097 (1.425)  45.198 (0.660)  34.029 (0.534)   
                 40     99.835 (1.402)  11.039 (0.164)  10.400 (0.164)   
                 200    99.631 (1.420)   2.199 (0.034)   2.182 (0.034)   
                 1000  100.301 (1.387)   0.445 (0.007)   0.445 (0.007)   
CI Coverage      10      0.951 (0.002)   0.920 (0.003)   0.924 (0.003)   
                 40      0.954 (0.002)   0.945 (0.002)   0.942 (0.002)   
                 200     0.951 (0.002)   0.954 (0.002)   0.955 (0.002)   
                 1000    0.951 (0.002)   0.950 (0.002)   0.953 (0.002)   
CI Nominal Level 10      0.950 (0.000)   0.950 (0.000)   0.950 (0.000)   
                 40      0.950 (0.000)   0.950 (0.000)   0.950 (0.000)   
                 200     0.950 (0.000)   0.950 (0.000)   0.951 (0.000)   
                 1000    0.950 (0.000)   0.950 (0.000)   0.952 (0.000)   
CI Length        10     39.210 (0.000)  24.535 (0.052)  20.534 (0.030)   
                 40     39.210 (0.000)  12.830 (0.019)  12.168 (0.016)   
                 200    39.210 (0.000)   5.828 (0.008)   5.771 (0.007)   
                 1000   39.210 (0.000)   2.611 (0.003)   2.615 (0.003)   

Methods                         BRI-A  
Metrics                                
Bias             10     0.440 (0.059)  
                 40     0.166 (0.032)  
                 200    0.047 (0.015)  
                 1000   0.016 (0.007)  
MSE              10    35.225 (0.579)  
                 40    10.477 (0.167)  
                 200    2.183 (0.034)  
                 1000   0.445 (0.007)  
CI Coverage      10     0.955 (0.002)  
                 40     0.952 (0.002)  
                 200    0.957 (0.002)  
                 1000   0.954 (0.002)  
CI Nominal Level 10     0.950 (0.000)  
                 40     0.950 (0.000)  
                 200    0.951 (0.000)  
                 1000   0.952 (0.000)  
CI Length        10    23.494 (0.036)  
                 40    12.675 (0.017)  
                 200    5.820 (0.007)  
                 1000   2.619 (0.003)

In [9]:
print(results_df.to_latex())

\begin{tabular}{llllll}
\toprule
 & Methods & Prior & DIM & LIB & BRI-A \\
Metrics &  &  &  &  &  \\
\midrule
\multirow[t]{4}{*}{Bias} & 10 & 0.062 (0.099) & -0.041 (0.067) & 0.406 (0.058) & 0.440 (0.059) \\
 & 40 & -0.143 (0.100) & -0.023 (0.033) & 0.155 (0.032) & 0.166 (0.032) \\
 & 200 & 0.130 (0.100) & -0.004 (0.015) & 0.047 (0.015) & 0.047 (0.015) \\
 & 1000 & 0.068 (0.100) & 0.006 (0.007) & 0.016 (0.007) & 0.016 (0.007) \\
\cline{1-6}
\multirow[t]{4}{*}{MSE} & 10 & 98.097 (1.425) & 45.198 (0.660) & 34.029 (0.534) & 35.225 (0.579) \\
 & 40 & 99.835 (1.402) & 11.039 (0.164) & 10.400 (0.164) & 10.477 (0.167) \\
 & 200 & 99.631 (1.420) & 2.199 (0.034) & 2.182 (0.034) & 2.183 (0.034) \\
 & 1000 & 100.301 (1.387) & 0.445 (0.007) & 0.445 (0.007) & 0.445 (0.007) \\
\cline{1-6}
\multirow[t]{4}{*}{CI Coverage} & 10 & 0.951 (0.002) & 0.920 (0.003) & 0.924 (0.003) & 0.955 (0.002) \\
 & 40 & 0.954 (0.002) & 0.945 (0.002) & 0.942 (0.002) & 0.952 (0.002) \\
 & 200 & 0.951 (0.002) & 0.954 (0.002